In [2]:
import sqlite3
import pandas as pd

In [89]:
follows = pd.read_csv('follows.csv')
users = pd.read_csv('users.csv')

In [93]:
follows.set_index('user_id', inplace=True)

In [95]:
users.set_index('user_id', inplace=True)

In [100]:
# 1. How many users are there in each house?
users.groupby('house').count().drop(columns=['last_name']).rename(columns={'first_name': 'count'})

,count
house,
Gryffindor,4
Hufflepuff,1
Ravenclaw,2
Slytherin,2


In [101]:
# 2. List all following links that were created before September 1st, 1993
follows[follows['date'] < '1993-09-01']

,follows,date
user_id,,
2,1,1989-01-01
3,1,1993-07-01
4,2,1988-08-08
4,1,1988-08-08
5,6,1986-01-10
7,1,1990-02-02


In [106]:
# Merge the two tables together
users_follows = users.merge(follows, left_index=True, right_index=True)

In [120]:
# 3. List all the following links established before September 1st 1993, but this time use the users first names

# Filter users_follows dataframe to only include rows before September 1993
users_sept = users_follows.loc[users_follows['date'] < '1993-09-01']

# Join first_name from users dataframe based on follows column in users_sept
users_sept = users_sept.merge(users[['first_name']], left_on='follows', right_on='user_id', how='left')

# Keep only the necessary columns
users_sept = users_sept[['first_name_x', 'first_name_y', 'date']]
users_sept.columns = ['first_name', 'follows', 'date']

In [121]:
users_sept

,first_name,follows,date
0,Ron,Harry,1989-01-01
1,Hermonie,Harry,1993-07-01
2,Ginny,Ron,1988-08-08
3,Ginny,Harry,1988-08-08
4,Draco,Tom,1986-01-10
5,Luna,Harry,1990-02-02


In [166]:
# 4. Give a count of how many people followed each user as of 1999-12-31. Give the result in term of "users full name, number of followers
users_dec = (
    users_follows[users_follows['date'] < '1999-12-31']
    .merge(users[['first_name']], left_on='follows', right_on='user_id', how='left')
    .groupby('first_name_y')
    .agg(number_of_followers=('follows', 'count'))
    .reset_index()
    .sort_values(by='number_of_followers', ascending=False)
    .rename(columns={'first_name_y': 'first_name'})
)

users_dec.columns = ['first_name', 'number_of_followers']

In [168]:
users_dec

,first_name,number_of_followers
3,Harry,5
0,Cedric,3
1,Cho,3
6,Ron,3
2,Ginny,1
4,Hermonie,1
5,Luna,1
7,Tom,1


In [179]:
# 5. List the number of users each user follows

users_follows.groupby('first_name').agg(number_of_follows=('follows', 'count')).reset_index().sort_values(by='number_of_follows', ascending=False)

,first_name,number_of_follows
4,Harry,5
3,Ginny,3
5,Hermonie,3
1,Cho,2
2,Draco,2
6,Luna,2
7,Ron,2
0,Cedric,1


In [197]:
# 6. List all rows from follows where someone from one house follows someone from a different house. Please give the result with user names. 
houses = users_follows.merge(users[['first_name', 'house']], left_on='follows', right_on='user_id', how='left')
houses = houses[houses['house_x'] != houses['house_y']]
houses = houses[['first_name_x', 'first_name_y', 'house_x', 'house_y']].rename(columns={'first_name_x': 'first_name', 'first_name_y': 'follows', 
                                                                                            'house_x': 'house', 'house_y':'follows_house'})

In [198]:
houses

,first_name,follows,house,follows_house
2,Harry,Draco,Gryffindor,Slytherin
3,Harry,Luna,Gryffindor,Ravenclaw
4,Harry,Cho,Gryffindor,Ravenclaw
9,Hermonie,Cedric,Gryffindor,Hufflepuff
12,Ginny,Cedric,Gryffindor,Hufflepuff
13,Draco,Harry,Slytherin,Gryffindor
15,Luna,Harry,Ravenclaw,Gryffindor
17,Cho,Harry,Ravenclaw,Gryffindor
18,Cho,Cedric,Ravenclaw,Hufflepuff
19,Cedric,Cho,Hufflepuff,Ravenclaw


In [231]:
# 7. List all unrequited followings (i.e. where A follows B but B does not follow A

unrequited = users_follows.merge(users[['first_name']], left_on='follows', right_on='user_id', how='left')
unrequited = unrequited[['first_name_x', 'first_name_y']]
unrequited['pair'] = unrequited['first_name_x'] + unrequited['first_name_y']
unrequited = unrequited[~unrequited['pair'].isin(unrequited['first_name_y'] + unrequited['first_name_x'])]
unrequited[['first_name_x', 'first_name_y']]

,first_name_x,first_name_y
7,Hermonie,Harry
9,Hermonie,Cedric
10,Ginny,Ron
12,Ginny,Cedric
14,Draco,Tom
16,Luna,Cho
